In [1]:
from re import search
import urllib
import urllib.request
import re
import _thread
import time
import sys
import requests
from bs4 import BeautifulSoup
defaultencoding = 'utf-8'
if sys.getdefaultencoding() != defaultencoding:
    reload(sys)
    sys.setdefaultencoding(defaultencoding)

In [54]:
class GHCrawler:
#initialization
    def __init__(self):
        self.pageIndex = 1
        self.user_agent = 'Mozilla/4.0 (compatible; MSIE 5.5; Windows NT)'
        #initialize headers
        self.headers = { 'User-Agent' : self.user_agent }
        #every pages' stories
        self.stories = []
        #if continue
        self.enable = False
        
    #get pages' code
    def getPage(self,pageIndex,language_type,search_content):
        #print project_number,sort_options,search_content+"========="
        try:
            #url = 'http://www.qiushibaike.com/hot/page/' + str(pageIndex)
            #url = 'https://github.com/search?o=desc&q=big+data&s=stars&p='+str(pageIndex)
            # url = 'https://github.com/search?'+str(sort_options)+'&q='+str(search_content)+'&p='+str(pageIndex)
            url = 'https://github.com/search?l='+str(language_type)+'&p='+str(pageIndex)+'&q='+str(search_content)+'&type=Code'
            # https://github.com/search?l=Java&p=3&q=search+engine&type=Code
            print(url)
            #construct request
            request = urllib.request.Request(url,headers = self.headers)
            #urlopen to get codes
            response = urllib.request.urlopen(request)
            #UTF-8 transform
            pageCode = response.read().decode('utf-8')
            return pageCode
        except urllib.request.URLError:
            print("Connecting github failed")
            return None
        
    def getPage_login(self,pageIndex,language_type,search_content):
        r1 = requests.get(
        url='https://github.com/login')
        s1 = BeautifulSoup(r1.text, 'html.parser')
        token = s1.find(name='input', attrs={'name': 'authenticity_token'}).get('value')
        r1_cookie_dict = r1.cookies.get_dict()
        r2 = requests.post(
        url='https://github.com/session',
        data={
        'commit':'Sign in',
        'utf8':'✓',
        'authenticity_token':token,
        'login':'630853543@qq.com',
        'password':'***'  
        },
        cookies=r1_cookie_dict
        )
        r2_cookie_dict = r2.cookies.get_dict()
        url='https://github.com/search?l='+str(language_type)+'&p='+str(pageIndex)+'&q='+str(search_content)+'&type=Code'
        r3 = requests.get(
        url,
        cookies=r2_cookie_dict
        )
        print(url)
        pageCode = r3.text
        return pageCode
        
        
        
    def getPageItems(self,pageIndex,language_type,search_content):
        #print project_number,sort_options,search_content+"===="
        pageCode = self.getPage_login(pageIndex,language_type,search_content)
        #print pageCode
        if not pageCode:
            print("Page loading failed....")
            return None
        
        # reg = '&quot;url&quot;:&quot;.+?&quot;},&quot;originating_url&quot;:&quot;'
        reg = '<div class="f4 text-normal">.+?<div class="file-box blob-wrapper my-1">'
        pattern = re.compile(reg,re.S)
        items = re.findall(pattern,pageCode)
        
        urls = []
        for item in items:
            soup = BeautifulSoup(item, 'html.parser')
            urls.append('https://raw.githubusercontent.com'+soup.find('a')['href'].replace('blob/',''))
            # urls.append(item.strip('&quot;url&quot;:&quot;').strip('&quot;},&quot;originating_'))
        
        return urls
    
    def loadPage(self,pageIndex,language_type,search_content):
        #print project_number,sort_options,search_content
        pageUrls = self.getPageItems(self.pageIndex,language_type,search_content)
        for pageurl in pageUrls:
            self.stories.append(pageurl)
            
    def getCode(self, url):
        c = requests.get(url)
        code = c.text
        #request = urllib.request.Request(url,headers = self.headers)
        #urlopen to get codes
        #response = urllib.request.urlopen(request)
        #code = response.read().decode('gdk')
        return code
    
    def start_crawler(self,page_number,language_type,search_content):
        print(u"Reading github pages，Search for "+search_content)
        #print project_number,sort_options,search_content
        self.enable = True
        while self.pageIndex <= page_number:
            self.loadPage(self.pageIndex,language_type,search_content)
            self.pageIndex += 1
        print('\n')
        count = 1
        for i in self.stories:
            print(i)
            codefile = self.getCode(str(i))
            f = open('./data/Code %d.txt'%count,'wt',encoding='utf-8') #output.txt - 文件名称及格式 w - writing
            f.write(codefile)
            f.close()
            count = count+1

In [55]:
spider = GHCrawler()

In [56]:
project_number = 20
page_number = 2
#0:most stars,1:fewest stars,2:most forks,3:fewest forks,4:recently updated,5:least recently updated
options = 0
language_type = 'Java'
print('language type: '+language_type)
#search content
content = "search engine"
search_content = content.replace(" ", "+")
spider.start_crawler(page_number,language_type,search_content)

language type: Java
Reading github pages，Search for search+engine
https://github.com/search?l=Java&p=1&q=search+engine&type=Code
https://github.com/search?l=Java&p=2&q=search+engine&type=Code


https://raw.githubusercontent.com/qiaohhgz/hotel/648bf684a053b3afda0edc531b77038420fc4928/src/main/java/hotel/common/constant/SearchEngineIDT.java
https://raw.githubusercontent.com/haytham-salhi/Spread/a3793a45be6c6fd2a327627b71d3b693cfc4b5a1/src/main/java/com/spread/persistence/rds/repository/SearchResultRepository.java
https://raw.githubusercontent.com/OptiPop/packages_apps_Browser/fd3a31588f283de64309895170f5ce29039a60b4/src/com/android/browser/search/SearchEngines.java
https://raw.githubusercontent.com/Macheal2/Browser/b8ac33371093297a1466989264e7877bae9a3340/src/com/android/browser/search/SearchEngines.java
https://raw.githubusercontent.com/liferay/liferay-central-test-2/92d21ade0e059deb1bb0cf5cef5e8b9ddf5d3637/portal-kernel/src/com/liferay/portal/kernel/search/AbstractSearchEngineConfigura